# A4: Authorship

In [264]:
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, sent_tokenize
from nltk import ngrams, pos_tag
import nltk
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
from sklearn.svm import SVC
import numpy as np
import string

In [267]:
# Download the 'stopwords' and 'punkt' from the Natural Language Toolkit, you can comment the next lines if already present.
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nilsk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nilsk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nilsk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [5]:
# Load the dataset into memory from the filesystem
def load_data(dir_name):
    return sklearn.datasets.load_files('data/%s' % dir_name, encoding='utf-8')


def load_train_data():
    return load_data('train')


def load_test_data():
    return load_data('test')

In [13]:
# Classify using the features
def classify(train_features, train_labels, test_features):
    clf = SVC(kernel='linear')
    clf.fit(train_features, train_labels)
    return clf.predict(test_features)

In [14]:
# Evaluate predictions (y_pred) given the ground truth (y_true)
def evaluate(y_true, y_pred):
    # TODO: What is being evaluated here and what does it say about the performance? Include or change the evaluation
    # TODO: if necessary.
    # A: y_true is the actual author, while y_pred is the predicted author by the program. These values are fairly 
    # representative about the quality of a classifier and the features.
    recall = sklearn.metrics.recall_score(y_true, y_pred, average='macro')
    print("Recall: %f" % recall)

    precision = sklearn.metrics.precision_score(y_true, y_pred, average='macro')
    print("Precision: %f" % precision)

    f1_score = sklearn.metrics.f1_score(y_true, y_pred, average='macro')
    print("F1-score: %f" % f1_score)

    return recall, precision, f1_score

In [279]:
# Extract features from a given text
def extract_features(text):
    bag_of_words = [x for x in wordpunct_tokenize(text)]
    sentences = [x for x in sent_tokenize(text)]

    features = []
    # Example feature 1: count the number of words
    features.append(len(bag_of_words))

    # Example feature 2: count the number of words, excluded the stopwords
    no_stopwords = [x for x in bag_of_words if x.lower() not in stop_words]
    features.append(len(no_stopwords))

    # TODO: Follow the instructions in the assignment and add your own features.
    
    # Feature 3-5: take top k most frequent word counts
    k = 3
    freq_dict = {i:no_stopwords.count(i) for i in set(no_stopwords)}
    features.extend(sorted(freq_dict.values(), reverse=True)[:k])
    
    # Feature 6: average word length
    word_lengths = [len(x) for x in no_stopwords]
    features.append(sum(word_lengths) / len(word_lengths))
    
    # Feature 7: number of stopwords
    features.append(len([x for x in bag_of_words if x.lower() in stop_words]))
    
    # Feature 8: average sentence length
    sentence_lengths = [len(x) for x in sentences]
    features.append(sum(sentence_lengths) / len(sentence_lengths))
    
    # Feature 9-23: top m most frequent characters
    m = 14
    chars = list(text)
    features.extend(sorted({i:chars.count(i) for i in set(chars)}.values(), reverse=True)[:m])
    
    # Feature 24-26: most frequent n-grams
    n = 3
    n_grams = []
    for gram in ngrams(bag_of_words, n):  # I hate this
        n_grams.append(gram)
    features.extend(sorted({i:n_grams.count(i) for i in set(n_grams)}.values(), reverse=True)[:n])
    
    # Feature 27: Punctuation
    features.append(len(text.translate(str.maketrans('', '', string.punctuation))))
    
    # Feature 28: Number of sentences
    features.append(len(sentences))
    
    # Feature 29-129: Function words frequency
    function_words="a between in nor some upon about both including nothing somebody us above but inside of someone used after by into off something via all can is on such we although cos it once than what am do its one that whatever among down latter onto the when an each less opposite their where and either like or them whether another enough little our these which any every lots outside they while anybody everybody many over this who anyone everyone me own those whoever anything everything more past though whom are few most per through whose around following much plenty till will as for must plus to with at from my regarding toward within be have near same towards without because he need several under worth before her neither she unless would behind him no should unlike yes below i nobody since until you beside if none so up your"
    function_words = function_words.split(" ")
    p = 100
    frequencies = []
    for i in range(p):
        key = function_words[i]
        frequencies.append(freq_dict.get(function_words[i]) if key in freq_dict else 0)
    features.extend(frequencies)
    
    # Feature 130: Parantheses
    features.append(freq_dict.get('(') if '(' in freq_dict else 0)
    
    # Feature 131: Question marks
    features.append(freq_dict.get('?') if '?' in freq_dict else 0)
    
    # Feature 132-155: Pos tag frequencies
    pos_tags = [y for x, y in nltk.pos_tag(bag_of_words)]
    q = 23
    features.extend(sorted({i:pos_tags.count(i) for i in set(pos_tags)}.values(), reverse=True)[:q])
    
    return features

In [280]:
train_data = load_train_data()

# Extract the features
features = list(map(extract_features, train_data.data))

# Classify and evaluate
skf = sklearn.model_selection.StratifiedKFold(n_splits=10)
scores = []
for fold_id, (train_indexes, validation_indexes) in enumerate(skf.split(train_data.filenames, train_data.target)):
    # Print the fold number
    print("Fold %d" % (fold_id + 1))

    # Collect the data for this train/validation split
    train_features = [features[x] for x in train_indexes]
    train_labels = [train_data.target[x] for x in train_indexes]
    validation_features = [features[x] for x in validation_indexes]
    validation_labels = [train_data.target[x] for x in validation_indexes]

    # Classify and add the scores to be able to average later
    y_pred = classify(train_features, train_labels, validation_features)
    scores.append(evaluate(validation_labels, y_pred))

    # Print a newline
    print("")

# Print the averaged score
recall = sum([x[0] for x in scores]) / len(scores)
print("Averaged total recall", recall)
precision = sum([x[1] for x in scores]) / len(scores)
print("Averaged total precision", precision)
f_score = sum([x[2] for x in scores]) / len(scores)
print("Averaged total f-score", f_score)

# TODO: Once you are done crafting your features and tuning your model, also test on the test set and report your
# TODO: findings. How does the score differ from the validation score? And why do you think this is?
# test_data = load_test_data()
# test_features = list(map(extract_features, test_data.data))
#
# y_pred = classify(features, train_data.target, test_features)
# evaluate(test_data.target, y_pred)

Fold 1
Recall: 0.355556


D:\Programs\Anaconda\envs\TxMM\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.248889
F1-score: 0.258519

Fold 2
Recall: 0.366667
Precision: 0.315238
F1-score: 0.323316

Fold 3


D:\Programs\Anaconda\envs\TxMM\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.277778
Precision: 0.237037
F1-score: 0.247407

Fold 4


...

D:\Programs\Anaconda\envs\TxMM\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.477778
Precision: 0.383333
F1-score: 0.401481

Fold 10


D:\Programs\Anaconda\envs\TxMM\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.411111
Precision: 0.337037
F1-score: 0.357778

Averaged total recall 0.4
Averaged total precision 0.31430158730158725
Averaged total f-score 0.3327971781305114


D:\Programs\Anaconda\envs\TxMM\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best so far: 0.333 - 20

Notes:
type-token ratio (vocabulary richness) actually decreased F-score
whitespace count didn't work, some weird error with isnan (unsafe casting)
alphabet frequency brings down f-score
number of characters brings down f-score
quotation didn't work
